## Prediction Using Support Vector Machines (SVM)
## Radial basis function kernel (RBF)

In [40]:
## Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn import metrics

#### Load dataset

In [41]:
url = "Test-Data-6.csv"
df = pd.read_csv(url)
include = [ 'Age', 'Gender', 'Religon', 'Race', 'Nature Of Occupation' ,
           'Civil Status' ,'Education Level' ,'Reason' ,'Lifetime Psychiatric Hospitalizations' ,
           'Past Suicide Attempts' ,'Any suicidal thoughts mentioned' ,'Self-Injurious Behaviour' ,
           'Psychiatric Disorders' ,'Past Illnesses' ,'Alcohol/ drug Consumption','Anger',
           'Sleep Problem', 'Social Iscolation', 'Sad/ Weary' , 'Humilated', 'Not Suicide(0)/Suicide(1)']  # Except the year, all the features are taken 
df1 = df[include]
print(df1.head(20))
print(df1.describe())

    Age Gender    Religon        Race  \
0    30      F      Islam      Muslim   
1    91      F      Islam      Muslim   
2    85      M      Islam      Muslim   
3    87      M  Christian  Sinhalese    
4    59      M      Islam      Muslim   
5    31      M      Other  Sinhalese    
6    75      M  Christian      Burger   
7    47      M   Buddhist  Sinhalese    
8    58      M  Christian      Burger   
9    36      M      Hindu       Tamil   
10   87      M   Buddhist  Sinhalese    
11   73      M  Christian      Burger   
12   44      M   Buddhist  Sinhalese    
13   62      M   Buddhist  Sinhalese    
14   21      M      Hindu       Tamil   
15   70      M   Buddhist  Sinhalese    
16   68      M      Hindu       Tamil   
17   43      F   Buddhist  Sinhalese    
18   16      M      Islam      Muslim   
19   35      M      Hindu       Tamil   

                                 Nature Of Occupation Civil Status  \
0   Administrative Executive Managerial & related ...    Unmarried  

#### Replacing unwanted labels with a unique label

In [42]:
df['Reason'].replace(['Asthma', "Alzheimer's disease", 'Diabetes',
              'Cirrhosis', 'Ischemic heart disease', 
              'Road Accident','Chronic kidney disease'],
             ['Natural Death','Natural Death','Natural Death','Natural Death',
              'Natural Death','Natural Death','Natural Death'], inplace = True)
df['Reason'].replace(['Stroke/Chronic diseases & Physical disabilities'],
             ['Chronic diseases & Physical disabilities'], inplace = True)
df1 = df[include]

#### Data Preprocessing

In [43]:
categoricals = []
for col, col_type in df1.dtypes.iteritems():
     if col_type == 'O':
          categoricals.append(col)
     else:
          df1[col].fillna(0, inplace=True)

c:\users\dinis\pycharmprojects\suicide-risk-predictor\venv\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


#### LabelEncoder

In [44]:
category_col =['Gender', 'Religon', 'Race', 'Nature Of Occupation' ,
           'Civil Status' ,'Education Level' ,'Reason' ,
           'Psychiatric Disorders' ,'Past Illnesses' ,'Alcohol/ drug Consumption'] 
labelEncoder = preprocessing.LabelEncoder()
mapping_dict={}
for col in category_col:
    df1[col] = labelEncoder.fit_transform(df1[col])
    le_name_mapping = dict(zip(labelEncoder.classes_, labelEncoder.transform(labelEncoder.classes_)))
    mapping_dict[col]=le_name_mapping
print(df1.head(20))

for x, y in mapping_dict.items():
  print(x, y)

    Age  Gender  Religon  Race  Nature Of Occupation  Civil Status  \
0    30       0        3     1                     0             2   
1    91       0        3     1                     8             1   
2    85       1        3     1                     9             1   
3    87       1        1     3                    12             1   
4    59       1        3     1                     4             2   
5    31       1        4     3                     1             2   
6    75       1        1     0                     2             1   
7    47       1        0     3                    12             1   
8    58       1        1     0                     2             2   
9    36       1        2     4                     2             1   
10   87       1        0     3                    12             1   
11   73       1        1     0                     3             1   
12   44       1        0     3                     1             1   
13   62       1     

c:\users\dinis\pycharmprojects\suicide-risk-predictor\venv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


##### Split the dataset

In [45]:
X = df1.values[:, 0:20]
Y = df1.values[:,20]
print(X)
print(Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state=109)  # 70% training and 30% testing

[[30  0  3 ...  0  0  0]
 [91  0  3 ...  0  0  0]
 [85  1  3 ...  0  0  0]
 ...
 [15  1  0 ...  0  1  1]
 [75  1  3 ...  0  0  0]
 [56  0  0 ...  1  1  0]]
[0 0 0 1 0 1 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 0 1 1 1 1 0 1 0 1 0 0 1
 1 1 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0 1 0 1 0 1 1
 1 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 0 0 0
 0 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1 0 0 1 1 0 1 1 0 0 1
 0 0 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 1 0 0 1 1
 1 0 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0
 1 0 1 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1
 0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 0 0 0 0 1 0 0 1 0 1 1 1 0 0 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 0 0 1 0 0 1 1 1 1 1 1 1 1 0 1 0 0 0 1 0 1 1 0 1 1 1 1 1 1 1
 1 1 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1
 0 1 0 0 1 1 0 1 1 0 0 1 0 1 0 1 0 0 1 0 0 1 1 1 0 1 1 1 1 0 1 1 0 0 1 0 0
 0 0 0 1 0 1 1 0 0 

#### Model Generation

In [46]:
#Importing the svm model
from sklearn import svm

#Build a svm Classifier
clf = svm.SVC(kernel='rbf') # RBF Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)


#### Finding the accuracy

In [47]:
print("Accuracy from SVM(RBF) :",metrics.accuracy_score(y_test, y_pred))

Accuracy from SVM(RBF) : 0.81


#### Precision and Recall

In [48]:
print("Precision from SVM(RBF):",metrics.precision_score(y_test, y_pred))
print("Recall from SVM(RBF):",metrics.recall_score(y_test, y_pred))

Precision from SVM(RBF): 0.782608695652174
Recall from SVM(RBF): 0.8513513513513513


In [49]:
import pickle

#creating and training a model
#serializing our model to a file called model.pkl
pickle.dump(clf, open("model.pkl","wb"))
print("Model was created!")

Model was created!


In [50]:
print(clf.predict([[31,0,4,11,2,6,0,7,1,1,0,0,3,9,2,0,0,0,0,0]]))

[1]


In [52]:
print(clf.predict([[31,0,4,11,2,6,0,7,1,1,1,0,3,9,2,0,0,0,0,0]]))


In [52]:
print(clf.predict([[31,0,4,11,2,6,0,7,1,1,1,1,3,9,2,0,0,0,0,0]]))


In [52]:

print(clf.predict([[31,0,4,11,2,6,0,7,1,1,1,1,3,9,2,0,0,0,0,0]]))